TECH CHALLENGE - FASE I


    TAREFAS:
        Exploração de dados:
            Carregue a base de dados e explore suas características.
            Analise estatísticas descritivas e visualize distribuições relevantes.

        Pré-processamento de dados:
            Realize a limpeza dos dados, tratando valores ausentes (se necessário).
            Converta variáveis categóricas em formatos adequados para modelagem.

        Modelagem:
            Crie um modelo preditivo de regressão utilizando uma técnica à sua escolha (por exemplo, Regressão Linear, Árvores de Decisão, etc).
            Divida o conjunto de dados em conjuntos de treinamento e teste.

        Treinamento e avaliação do modelo:
            Treine o modelo com o conjunto de treinamento.

        Validação estatística:
            Utilize métricas estatísticas para validar a eficácia do modelo (p-value, intervalos de confiança).

        O que avaliaremos:
            Apresente resultados visuais, como gráficos de previsões vs. valores reais.
            Elabore um relatório que inclua uma análise dos resultados, insights obtidos e validação estatística.

        Observações:
            Esperamos que o modelo seja capaz de fazer previsões confiáveis dos custos médicos individuais com base nas características fornecidas.



ENTREGÁVEL:

    Como entregável, o grupo deve enviar um vídeo junto com o link do github do projeto e o código desenvolvido, apresentando o passo a passo do que foi utilizado como a fonte de dados e como os modelos foram criados.

    O vídeo deve estar em uma plataforma como Youtube.

    Boa sorte! Qualquer dúvida, não deixe de nos chamar no Discord, lá poderemos tirar suas dúvidas e auxiliar nesta tarefa.



* Créditos da base de dados completa (com maior volume de dados):
https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning



In [31]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [53]:
# Nome da base de dados
NOME_BASE_DE_DADOS = "../Base_de_dados/insurance.csv"
# NOME_BASE_DE_DADOS = "../Base_de_dados/insurance_dataset.csv"

Montagem do da base de dados via google drive:

In [30]:
# from google.colab import drive
# drive.mount('/content/drive/')

# /content/drive/MyDrive/insurance.csv

Realizando a remoção dos itens repetidos manualmente

In [54]:
# Função para a remoção de duplicidades em arquivo de dados (linha a linha):
def remocao_manual_duplicidades():
    novo_conteudo = ""

    lista_linhas_repetidas = []

    with open(NOME_BASE_DE_DADOS, 'r') as arquivo:
        contador_linha = 0
        for linha in arquivo:
            contador_linha = 0
            with open(NOME_BASE_DE_DADOS, 'r') as arquivo2:
                for linha2 in arquivo2:
                    if linha == linha2:
                        contador_linha += 1
                        if(contador_linha > 1):
                            print(f"linha Repetida {linha2}")
                            lista_linhas_repetidas.append(linha)
                        elif(contador_linha == 1):
                            if(linha not in lista_linhas_repetidas):
                                novo_conteudo += linha

    with open(NOME_BASE_DE_DADOS, 'w') as arquivo:
        arquivo.write(novo_conteudo)


if("insurance_dataset.csv" not in NOME_BASE_DE_DADOS):
    remocao_manual_duplicidades()
    

In [65]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# para deixar todas as saídas com os mesmos valores obtidos na live.
import numpy as np
np.random.seed(42)

# Importando a base de dados
dados_original = pd.read_csv(NOME_BASE_DE_DADOS, sep=",")

# Créditos para a base de dados importada do projeto: https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning
if("insurance_dataset.csv" in NOME_BASE_DE_DADOS):
    # dados_original = pd.read_csv("../Base_de_dados/insurance_dataset.csv", sep=",")
    # Retirando as colunas não utilizadas no Tech Challenge
    dados_original.drop(['medical_history'], axis=1, inplace=True)
    dados_original.drop(['family_medical_history'], axis=1, inplace=True)
    dados_original.drop(['exercise_frequency'], axis=1, inplace=True)
    dados_original.drop(['occupation'], axis=1, inplace=True)
    dados_original.drop(['coverage_level'], axis=1, inplace=True)
    # Renomeando a coluna 'B' para 'Coluna B'
    dados_original.rename(columns={'gender': 'sex'}, inplace=True)

Analisando o conteúdo dos dados

In [66]:
dados_original.head()

,age,sex,bmi,children,smoker,region,charges
0,46,male,21.45,5,yes,southeast,20460.307669
1,25,female,25.38,2,yes,northwest,20390.899218
2,38,male,44.88,2,yes,southwest,20204.476302
3,25,male,19.89,0,no,northwest,11789.029843
4,49,male,38.21,3,yes,northwest,19268.309838


In [67]:
# Informações estruturais sobre o dataframe
dados_original.shape


(1000000, 7)

In [68]:
# Estatísticas sobre o dataframe
display(dados_original.describe())

,age,bmi,children,charges
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,41.495282,34.001839,2.499886,16735.117481
std,13.855189,9.231680,1.707679,4415.808211
min,18.000000,18.000000,0.000000,3445.011643
25%,29.000000,26.020000,1.000000,13600.372379
50%,41.000000,34.000000,2.000000,16622.127973
75%,53.000000,41.990000,4.000000,19781.465410
max,65.000000,50.000000,5.000000,32561.560374


Verificando se existem valores nulos nas informações.

In [69]:
# Verificando a ausência de informações
dados_original.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [70]:
print("Informações sobre o DataFrame dados:")
print(dados_original.info())

Informações sobre o DataFrame dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   age       1000000 non-null  int64  
 1   sex       1000000 non-null  object 
 2   bmi       1000000 non-null  float64
 3   children  1000000 non-null  int64  
 4   smoker    1000000 non-null  object 
 5   region    1000000 non-null  object 
 6   charges   1000000 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 53.4+ MB
None


Feature scalling dos dados.

In [71]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

dados=dados_original.copy()

# Ajustar e transformar os rótulos
dados['sex'] = label_encoder.fit_transform(dados['sex'])
dados['smoker'] = label_encoder.fit_transform(dados['smoker'])
dados['region'] = label_encoder.fit_transform(dados['region'])

dados.head()

,age,sex,bmi,children,smoker,region,charges
0,46,1,21.45,5,1,2,20460.307669
1,25,0,25.38,2,1,1,20390.899218
2,38,1,44.88,2,1,3,20204.476302
3,25,1,19.89,0,0,1,11789.029843
4,49,1,38.21,3,1,1,19268.309838


Preparando os dados dos eixos X e Y para uso na padronização dos dados.

In [72]:
from sklearn.preprocessing import StandardScaler

features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region'] #, 'charges']
X = dados[features].values
y = dados['charges'].values

Padronização dos dados do eixo X.

In [ ]:
# X = StandardScaler().fit_transform(X)
# df_padronizado = pd.DataFrame(data=X, columns=features)
# display(df_padronizado.head())

Testando regressão linear múltipla

In [73]:
! pip install statsmodels

Treinando o modelo OLS e impmirindo as estatísticas de sua execução.

In [75]:
import statsmodels.api as sm

# Definir as variáveis independentes (X)
X = dados[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
# Definir a variável dependente (Y)
Y = dados['charges']

# Criar e ajustar o modelo de regressão linear múltipla
modelo_OLS = sm.OLS(Y, X).fit()

# Imprimir os resultados do modelo
print(modelo_OLS.summary())

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.944
Model:                            OLS   Adj. R-squared (uncentered):              0.944
Method:                 Least Squares   F-statistic:                          2.831e+06
Date:                Tue, 14 May 2024   Prob (F-statistic):                        0.00
Time:                        17:20:15   Log-Likelihood:                     -9.7331e+06
No. Observations:             1000000   AIC:                                  1.947e+07
Df Residuals:                  999994   BIC:                                  1.947e+07
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Testando a predição com base nos valores treinados.

In [76]:
#                        ['age', 'sex', 'bmi' , 'children', 'smoker', 'region']
print(modelo_OLS.predict([41   , 1    , 27.755, 0         , 0       , 1       ]))
print(modelo_OLS.predict([41   , 1    , 18.5  , 0         , 0       , 1       ]))


[11652.64038862]
[9807.35987414]


Imprimindo as estatísticas do modelo OLS.

In [77]:
! pip install scikit-learn
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# realizar as previsões
y_pred_OLS = modelo_OLS.predict(X)

# Calcular MAE, MSE e RMSE
mae_OLS = mean_absolute_error(Y, y_pred_OLS)
mse_OLS = mean_squared_error(Y, y_pred_OLS)
rmse_OLS = np.sqrt(mse_OLS)

# Imprimir as métricas
print(f'MAE: {mae_OLS}')
print(f'MSE: {mse_OLS}')
print(f'RMSE: {rmse_OLS}')

MAE: 3282.950123423186
MSE: 16657791.906269347
RMSE: 4081.395828178069


Treinando o modelo com o SKLearn

In [80]:
from sklearn.model_selection import train_test_split

dados_rlm = dados.copy()
# dados_rlm = df_padronizado.copy()

X_train_rlm, X_test_rlm, y_train_rlm, y_test_rlm = train_test_split(X,Y, test_size = 0.2, random_state=42)

Utilizando o modelo de regressão linear.

In [81]:
from sklearn.linear_model import LinearRegression

modelo_rlm = LinearRegression()
modelo_rlm.fit(X_train_rlm, y_train_rlm)

y_predict_rlm = modelo_rlm.predict(X_test_rlm)

# Calculando o R2
r2_rlm = modelo_rlm.score(X_test_rlm, y_test_rlm)

# Calculando o MSE
mse_rlm = mean_squared_error(y_test_rlm, y_predict_rlm)

import math

# Calculando o RMSE
rmse_rlm = math.sqrt(mse_rlm)

# Imprimindo os resultados
print("R²:", r2_rlm)
print("MSE:", mse_rlm)
print("RMSE:", rmse_rlm)



R²: 0.3573639690951279
MSE: 12506161.131693264
RMSE: 3536.405114193404


In [82]:
# Acessando os coeficientes do modelo
print("Coeficientes:", modelo_rlm.coef_)

Coeficientes: [  20.03907051 1003.40507722   49.70447993  198.51946093 5003.30310485
 -223.10046102]


Predizendo os valores conforme os dados abaixo:

In [83]:
#                         ['age', 'sex', 'bmi' , 'children', 'smoker', 'region']
print(modelo_rlm.predict([[41   , 1    , 27.755, 0         , 0       , 1       ]]))
print(modelo_rlm.predict([[41   , 1    , 18.5  , 0         , 0       , 1       ]]))
# array([198895.44142703])

[14027.39367878]
[13567.37871703]


/home/silas/PythonWorkspace/virtualenv/virtualenvPython3_FIAP_IA_Para_Devs/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/silas/PythonWorkspace/virtualenv/virtualenvPython3_FIAP_IA_Para_Devs/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
